In [ ]:
import numpy as np

### To Be Added

0. ~~Subtraction backward pass~~
1. ~~Log forward and backward~~
2. ~~Exponential forward and backward~~
3. ~~make tensors iterable~~
4. ~~make tensors subscriptable~~
5. ~~tensor.zeros~~
6. ~~tensor.ones~~
7. ~~tensor.normal~~
8. ~~tensor.eye~~
9. sigmoid forward and backward
10. softax forward and backward


In [ ]:
class Tensor:
    def __init__(self, data, _children=()):
        self.data = data if isinstance(data, np.ndarray) else np.array(data)
        # Set data to floats to division can be done
        self.data = self.data.astype(float) 
        self._prev = set(_children)
        self.grad = np.zeros_like(self.data)
        self.shape = self.data.shape
        self._backward = lambda: None
    
    def __add__(self, other):
        # Elementwise addition. Tensors must be the same size, or one of 
        # them must be a scalar 
        other = other if isinstance(other, Tensor) else Tensor(other)
        out = Tensor(self.data + other.data, (self, other))

        def _backward():
            self.grad += out.grad
            other.grad += out.grad
        out._backward = _backward

        return out
    
    def __sub__(self, other):    
        other = other if isinstance(other, Tensor) else Tensor(other)
        out = Tensor(self.data - other.data, (self, other))

        def _backward():
            self.grad += out.grad
            other.grad -= out.grad
        out._backward = _backward

        return out

    def __mul__(self, other):
        # Elementwise multiplication. Tensors must be the same size, or one of 
        # them must be a scalar 
        other = other if isinstance(other, Tensor) else Tensor(other)
        out = Tensor(self.data * other.data, (self, other))

        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward

        return out
    
    def __pow__(self, other):
        # The exponent must be a scalar
        assert isinstance(other, (int, float)), "Exponent must be a scalar (int/float)"
        out = Tensor(self.data ** other, (self, other))

        def _backward():
            self.grad += (other * self.data ** (other -1)) * out.grad
        out._backward = _backward

        return out
    
    def __matmul__(self, other):
        # Rows of self must match columns of other
        other = other if isinstance(other, Tensor) else Tensor(other)
        out = Tensor(self.data @ other.data, (self, other))

        def _backward():
            self.grad += out.grad @ np.transpose(other.data)
            other.grad += np.transpose(out.grad @ self.data)
        out._backward = _backward

        return out
    
    def relu(self):
        out = self.data * (self.data > 0)
        out = Tensor(self.data * (self.data > 0), (self,))

        def _backward():
            self.grad += (self.data > 0) * out.grad
        out._backward = _backward

        return out
    
    def transpose(self):
        out = Tensor(np.transpose(self.data), (self,))
        
        def _backward():
            self.grad += np.transpose(out.grad)
        out._backward = _backward
        
        return out
    
    def log(self):
        out = Tensor(np.log(self.data), (self,))

        def _backward():
            self.grad += (1/self.data) * out.grad
        out._backward = _backward

        return out
    
    def exp(self):
        out = Tensor(np.exp(self.data), (self,))
        
        def _backward():
            self.grad += np.exp(self.data) * out.grad
        out._backward = _backward

        return out
    
    def backward(self):
        # https://github.com/karpathy/micrograd/blob/master/micrograd/engine.py
        # topological order all of the children in the graph
        
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        build_topo(self)

        # go one variable at a time and apply the chain rule to get its gradient
        self.grad = np.ones_like(self.data)
        for v in reversed(topo):
            v._backward() 
        
    @classmethod
    def zeros(cls, shape):
        assert isinstance(shape, int) or isinstance(shape, tuple), f'shape should be int or tuple insted of {type(shape)}'
        return cls(np.zeros(shape))

    @classmethod
    def ones(cls, shape):
        assert isinstance(shape, int) or isinstance(shape, tuple), f'shape should be int or tuple insted of {type(shape)}'
        return cls(np.ones(shape))
    
    @classmethod
    def normal(cls, mean=0.0, std=1.0, shape=None):
        assert isinstance(shape, int) or isinstance(shape, tuple), f'shape should be int or tuple insted of {type(shape)}'
        return cls(np.random.normal(mean, std, shape))
    
    @classmethod
    def eye(cls, N, M=None):
        return cls(np.eye(N, M))

    def __neg__(self):
        return self * -1
    
    def __radd__(self, other):
        return self + other
    
    def __rsub__(self, other):
        return other - self
    
    def __rmul__(self, other):
        return self * other
    
    def __truediv__(self, other):
        return self * other**-1
    
    def __rtruediv__(self, other):
        return other * self**-1
    
    def __repr__(self):
        return f'Tensor(data={self.data})'
    
    def __len__(self):
        return len(self.data)
    
    def __iter__(self):
        # Track the current element in the iterable
        self.current = 0
        return self
    
    def __next__(self):
        if self.current >= len(self.data):
            raise StopIteration
        current = self.data[self.current]
        self.current += 1
        return current
    
    def __getitem__(self, key):
        return self.data[key]
    
    def __setitem__(self, key, value):
        self.data[key] = value

## Test Basic Ops

In [ ]:
a = Tensor([1, 2, 3])
b = Tensor([4, 5, 6])
c = a + b
print(c)
d = Tensor([5, 3.0, 2.7])
e = d/c
print(e)

In [ ]:
a = Tensor([16.0, 2.0, 4.5])
b = Tensor([2.0, 4.0, 3.5])
c = a * b
print(c)

In [ ]:
a = Tensor([[2.0, 4.0, 5.0], 
           [1.0, 5.5, 2.4]])
b = 3.0
c = a ** b
print(c)

In [ ]:
a = Tensor([[-2.0, 4.0, 5.0], 
           [1.0, -5.5, 2.4]])
a.relu()

In [ ]:
a = Tensor([2.5])
b = Tensor([3.0])
c = a @ b
print(c)

In [ ]:
a = Tensor([[-2.0, 4.0, 5.0], 
           [1.0, -5.5, 2.4]])
b = Tensor([[-2.0, 4.0], 
           [1.0, 2.4],
           [1.4, 9.0]])
c = a @ b
print(a.shape, b.shape, c.shape)

In [ ]:
a = Tensor([[2.0, 4.0, 5.0], 
           [1.0, 5.5, 2.4]])
a.log()

In [ ]:
a = Tensor([[-2.0, 4.0, 5.0], 
           [1.0, -5.5, 2.4]])
print(np.exp(a.data))
print(a.exp())

## Test Autodiff

In [ ]:
x1 = Tensor([[8.0, 2.0]])
w1 = Tensor([[3.2, 1.2]])
z1 = w1 * x1 - x1
q1 = z1.relu()
y1 = q1 @ x1.transpose()
p1 = y1.log()
s1 = p1.exp()
p1.backward()

print(x1.grad)
print(w1.grad)
print(z1.grad)
print(q1.grad)
print(y1.grad)
print(p1.grad)

In [ ]:
import torch

x = torch.Tensor([[8.0, 2.0]]); x.requires_grad = True
w = torch.Tensor([[3.2, 1.2]]); w.requires_grad = True
z = w * x - x; z.retain_grad()
q = z.relu(); q.retain_grad()
y = q @ x.transpose(0, 1); y.retain_grad()
p = y.log(); p.retain_grad()
s = p.exp(); s.retain_grad()
p.backward()

print(x.grad)
print(w.grad)
print(z.grad)
print(q.grad)
print(y.grad)
print(p.grad)

## Test Iterable

In [ ]:
a = Tensor([[-2.0, 4.0, 5.0], 
           [1.0, -5.5, 2.4]])
for array in a:
    print(array)

In [ ]:
a[0], a[1]

In [ ]:
a[0] = np.array([3.8, 9.0, 2.3])

## Make tensors

In [ ]:
a = Tensor.zeros((2, 3))
a

In [ ]:
b = Tensor.ones((4, 2))
b

In [ ]:
c = Tensor.normal(shape=(5, 4))
c

In [ ]:
d = Tensor.eye(6)
d